# Decision Optimizerで数独を解く

## 問題データの定義

In [ ]:
# GRNG: Grid Range
GRNG = range(9)

In [ ]:
SUDOKU_PROBLEM_X = ( (0, 0, 0,  0, 1, 5,  0, 7, 0),
                     (6, 3, 0,  8, 0, 0,  0, 0, 0),
                     (0, 0, 8,  0, 4, 0,  0, 0, 0),
                     (0, 2, 5,  0, 0, 0,  0, 4, 0),
                     (3, 0, 0,  4, 7, 0,  2, 0, 0),
                     (1, 0, 0,  0, 0, 0,  0, 6, 0),
                     (8, 0, 0,  0, 0, 6,  0, 0, 0),
                     (0, 0, 0,  0, 2, 0,  0, 0, 0),
                     (0, 7, 2,  1, 0, 0,  0, 9, 0)
                    )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 表示用関数
def draw_grid(values):
    %matplotlib inline
    fig, ax = plt.subplots(figsize =(4,4))
    min_val, max_val = 0, 9
    R =  range(0,9)
    for l in R:
        for c in R:
            v = values[c][l]
            s = " "
            if v > 0:
                s = str(v)
            ax.text(l+0.5,8.5-c, s, va='center', ha='center')
        ax.set_xlim(min_val, max_val)
    ax.set_ylim(min_val, max_val)
    ax.set_xticks(np.arange(max_val))
    ax.set_yticks(np.arange(max_val))
    ax.grid()
    plt.show()

In [ ]:
# 問題の表示
draw_grid(SUDOKU_PROBLEM_X)

In [ ]:
# 問題をproblem変数に代入
problem = SUDOKU_PROBLEM_X

## CPLEXによる問題定義

In [ ]:
# ライブラリインポート
from docplex.cp.model import *

# モデルの生成
mdl = CpoModel(name="Sudoku")

In [ ]:
# 決定変数の定義

# 9 x 9 の配列に Clc という名前のCPLEX変数を定義します (C00, C01,.. C88)
# それぞれの変数は1から9までの整数値を取ります

grid = [[integer_var(min=1, max=9, name="C" + str(l) + str(c)) for l in GRNG] for c in GRNG]

In [ ]:
# 制約の定義

# 制約条件を定義していきます

# 同一行に同じ整数値をもってはいけない
# all_diff は「すべての要素が同じではいけない」という意味の制約を表現する関数です
for l in GRNG:
    mdl.add(all_diff([grid[l][c] for c in GRNG]))

# 同一列に同じ整数値をもってはいけない
for c in GRNG:
    mdl.add(all_diff([grid[l][c] for l in GRNG]))    
    
# 3 x 3 の矩形領域に同じ整数値があってはいけない
ssrng = range(0, 9, 3)
for sl in ssrng:
    for sc in ssrng:
        mdl.add(all_diff([grid[l][c] for l in range(sl, sl + 3) for c in range(sc, sc + 3)]))

In [ ]:
# 初期条件の設定

# C00からC88までのCPLEX変数に初期条件をとて与えられている値を設定していきます
# 設定は set_domainという関数で行います。
# 例えばマス目の値が7の場合該当する変数に対して 
# Cxx.set_domain(7, 7) (７以上7以下の値を設定) という設定を行います。

for l in GRNG:
    for c in GRNG:
        v = problem[l][c]
        if v > 0:
            grid[l][c].set_domain((v, v))
            # 設定した変数名と値の表示
            print(grid[l][c])

## CPLEXによる解の取得
これで準備は整いました。後はモデルのslove関数を呼び出すと数独の問題を解いてくれます。

In [ ]:
print('Solving model....')
msol = mdl.solve(TimeLimit=10)
print('Solved!')

In [ ]:
draw_grid(problem)
sol = [[msol[grid[l][c]] for c in GRNG] for l in GRNG]
print('Solve time: ',  msol.get_solve_time())
draw_grid(sol)